In [1]:
from copy import deepcopy
import os
import math
import shutil
import re
from glob import glob
from collections import defaultdict

import numpy as np
from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_colwidth', None)
import rispy

In [2]:
dump_dir = '../data/bntl-db-csv-dump-4March2024'

### Recensies

Link reviews to the book that they review.

In [3]:
publications = pd.read_csv(f"{dump_dir}/publications.csv", header=0, parse_dates=['creation_date', 'modification_date'])
publications['id'] = publications['id'].fillna('')
publications

,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
0,254444,NaN,artikeltijdschrift,2009,"Campenhout, Frans van. Ropsiennes, de nieuwe dichtbundel van Mark Meekers.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 14.","Over: Meekers, Mark. Ropsiennes: gedichten bij het leven en werk van Félicien Rops (1833-1898). Leuven: P, 2009.",NaN,NaN,NaN,NaN,19,2009-06-26 11:32:26,19.0,2009-06-26 11:45:26,campenhout_frans_van_ropsienne/campenhout_frans_van
1,254442,NaN,artikeltijdschrift,2009,"Campenhout, Frans van. Dichteres en romancière Liane Bruylants overleden.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 5-8.",NaN,NaN,NaN,NaN,NaN,19,2009-06-26 11:27:36,19.0,2009-06-26 11:46:40,campenhout_frans_van_dichteres/campenhout_frans_van
2,313784,NaN,artikeltijdschrift,2018,"Lievois, Katrien, Hanne Kloots & Noureddine Nahed. Kif-kif?: Arabische ontleningen in de Nederlandse, Spaanse en Arabische versie van ""Kiffe kiffe demain"".\r\nIn: Meertaligheid. Speciaal gedeelte van: Filter (Nijmegen): 25 (2018) 3 (sep) 37-47.","P. 46 Noten; p. 46 Bibliografie; p. 47 Bijlage: vertalingen van ""Kiffe kiffe demain"".",NaN,NaN,NaN,NaN,451,2023-11-29 16:36:19,451.0,2023-11-29 16:40:08,lievois_katrien_hanne_kloots_n/lievois_katrien
3,171436,862346606,artikeltijdschrift,1965,"Interprovinciale Prijs voor monografie aan een Westvlaming, [door] A.S.\r\nIn: West-Vlaanderen: 14 (1965) 79 (jan-feb) 66-67.","N.a.v.: Deblaere, Albert. De mystieke schrijfster Maria Petyt, (1623-1677). Gent: Secretarie der Academie, 1962.",NaN,166925.0,853271437,"vol. 14 (1965), afl. 79 (jan-feb), p. 66-67.",1,NaT,19.0,2009-06-26 10:25:49,interprovinciale_prijs_voor_mo
4,246627,NaN,artikelboek,2004,"Willaert, Frank. Margaret's booklets: memory in ""Vanden seven sloten"" by Jan van Ruusbroec.\r\nIn: Medieval memory: image and text; edited by Frank Willaert, Herman Braet, Tom Mertens, Theofiel Venckeleer. Turnhout: Brepols, 2004, p. 99-128.\r\n(Textes et études du Moyen Âge; 27).","Lezing, gehouden op het congres ""Memory in the Middle Ages"" aan de Universiteit Antwerpen, 7-9 maart 2002.",NaN,NaN,NaN,NaN,16,2008-06-18 12:05:19,16.0,2008-06-18 12:15:23,willaert_frank_margaret_s_book/willaert_frank
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302861,254432,NaN,recensie,1962,"Plard, Henri. Une mystique flamande retrouvée: Maria Petyt (1623-1677).\r\nIn: Études germaniques (Paris): 17 (1962) 3 (juill-sep) 347-354.",NaN,NaN,NaN,NaN,NaN,19,2009-06-26 10:31:29,18.0,2014-01-10 12:31:40,plard_henri_une_mystique_flama/plard_henri
302862,129705,NaN,recensie,2003,"Offermans, Cyrille. \r\nIn: Ons erfdeel (Rekkem): 46 (2003) 3 (juni) 446-448.",NaN,NaN,NaN,NaN,NaN,1,NaT,19.0,2012-12-06 10:04:44,cyrille_offermans_in_ons_erfde/offermans_cyrille_albert_willem_3
302863,129707,241847133,boek,2003,"Nederlands-Afghaanse grammatica = Nīdarlandī-Paśtū paśwayah / Sayed Nasir Ahmad (Sayyid Naṣīr Aḥmad). - 3e, herz. dr. - Utrecht : Nederlands Centrum Buitenlanders, cop. 2003. - XIV, 182 p. ; 24 cm Tekst in het Nederlands en het Pasjtoe. - Ook o.d.t.: Nederlands-Pashto grammatica. - Met lit. opg. ISBN 90-5517-169-7.",P. 181-182 Literatuur.\n1e-2e dr. ?,NaN,NaN,NaN,NaN,1,NaT,NaN,NaT,nederlands-afghaanse_grammatic/nasir_ahmad_sayed
302864,254436,NaN,recensie,1964,"Degroote, Gilbert.\r\nIn: Belgisch tijdschrift voor filologie en geschiedenis (Brussel): 42 (1964) 1, 152-153.",NaN,NaN,NaN,NaN,NaN,19,2009-06-26 10:37:20,18.0,2014-01-10 12:33:47,degroote_gilbert_in_revue_belg/degroote_gilbert


In [4]:
publications['title_description'] = [' '.join(p.split()).strip() for p in publications['title_description']]

In [5]:
publications[publications['title_description'] == 'Royen, P. Gerlach O.F.M. In: Levende talen: (1947) 190-191.']

,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
26,272188,NaN,recensie,1947,"Royen, P. Gerlach O.F.M. In: Levende talen: (1947) 190-191.",NaN,NaN,NaN,NaN,NaN,19,2011-10-06 13:32:35,19.0,2011-10-06 13:33:58,royen_p_gerlach_o_f_m_in_leven/royen_o_f_m_nicolaus_jacobus_hubertus


In [6]:
publications[publications['title_description'] == 'Caron, Willem Johannes Hubertus. Klank en teken bij Erasmus en onze oudste grammatici. Groningen [etc.]: Wolters, 1947. XII, 140 p. + stellingen.']

,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
255901,273,036356891,boek,1947,"Caron, Willem Johannes Hubertus. Klank en teken bij Erasmus en onze oudste grammatici. Groningen [etc.]: Wolters, 1947. XII, 140 p. + stellingen.","Proefschrift Vrije Universiteit Amsterdam.\r\nOok in: Caron, W. J. H. Klank en teken: verzamelde taalkundige studies; [voor herdr. gereedgemaakt en van een bibliografie voorz. door J. Knol; met medew. van H. J. Brinkman... et al.]. Groningen: Wolters-Noordhoff, 1972, p. 1-128.\r\nP. XII Lijst van gebruikte afkortingen; p. 141 Personenregister.\r\nO.a. ook over: Richard Dafforne, Anthonis de Huybert, Lambert ten Kate, Petrus Montanus, Arnold Moonen, H. L. Spiegel en Wigardus à Winschooten.",NaN,NaN,NaN,NaN,1,NaT,19.0,2011-10-06 13:37:17,klank_en_teken_bij_erasmus_en/caron_willem_johannes_hubertus


In [7]:
title2id = dict(zip(publications['title_description'], publications['id']))
id2title = dict(zip(publications['id'], publications['title_description']))

In [8]:
print(title2id['Royen, P. Gerlach O.F.M. In: Levende talen: (1947) 190-191.'])
print(title2id['Caron, Willem Johannes Hubertus. Klank en teken bij Erasmus en onze oudste grammatici. Groningen [etc.]: Wolters, 1947. XII, 140 p. + stellingen.'])

272188
273


In [9]:
print(id2title[272188])
print(id2title[273])

Royen, P. Gerlach O.F.M. In: Levende talen: (1947) 190-191.
Caron, Willem Johannes Hubertus. Klank en teken bij Erasmus en onze oudste grammatici. Groningen [etc.]: Wolters, 1947. XII, 140 p. + stellingen.


In [10]:
references = pd.read_csv(f'{dump_dir}/publication_publications.csv')
rev2ref = dict(zip(references['publication_id'], references['referenced_publication_id']))

In [11]:
print(rev2ref[272188])

273


In [12]:
llm_path = '../data/llm-dump'

for decade_folder in sorted(glob(f'{llm_path}/*')):
    print(':::', decade_folder, ':::')
    decade = decade_folder.split('/')[-1]

    for ris_path in sorted(glob(f'{decade_folder}/*_consolidated.ris')):
        print(ris_path)
        with open(ris_path) as f:
            entries = rispy.load(f, encoding='utf-8')
        for idx, entry in tqdm(list(enumerate(entries))):
            if not 'keywords' in entry or not 'recensie' in entry['keywords']:
                continue
            if 'notes_abstract' in entry:
                pub_title = ' '.join(entry['notes_abstract'].split()).strip()
            else:
                # this means that we weren't able to structure the item via the LLM...
                pub_title = ' '.join(entry['title'].split()).strip()
            
            ref_title = None
            try:
                #print('==============================')
                #print(pub_title)
                pub_id = title2id[pub_title]
                #print(pub_id)
                ref_id = rev2ref[pub_id]
                #print(ref_id)
                ref_title = id2title[ref_id]
                #print(ref_id, ref_title)
            except KeyError:
                pass
            if ref_title:
                entry['reviewed_item'] = ref_title
                entries[idx] = entry

        out_ris_path = ris_path.replace('.ris', f'_link_{decade}.ris')
        with open(out_ris_path, 'w') as bibliography_file:
            rispy.dump(entries, bibliography_file)

::: ../data/llm-dump/1940s :::
../data/llm-dump/1940s/JOUR_consolidated.ris


100%|██████████| 9897/9897 [00:00<00:00, 986455.33it/s]


::: ../data/llm-dump/1950s :::
../data/llm-dump/1950s/JOUR_consolidated.ris


100%|██████████| 6218/6218 [00:00<00:00, 1238669.31it/s]


::: ../data/llm-dump/1960s :::
../data/llm-dump/1960s/JOUR_consolidated.ris


...

100%|██████████| 20043/20043 [00:00<00:00, 1517143.44it/s]


::: ../data/llm-dump/2020s :::
../data/llm-dump/2020s/JOUR_consolidated.ris


100%|██████████| 5238/5238 [00:00<00:00, 1601294.78it/s]

::: ../data/llm-dump/2030s :::
::: ../data/llm-dump/misc :::
../data/llm-dump/misc/JOUR_consolidated.ris



100%|██████████| 3483/3483 [00:00<00:00, 2473964.58it/s]
